In [1]:
import os
import django
from django.conf import settings

# Set the Django settings module
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "sv_carbon_removal.settings")
django.setup()


In [2]:
import ee
import geemap
os.getcwd()

'/home/mfirdaus/gee_mapbox/backend/sv_carbon_removal'

In [3]:
# Trigger the authentication flow. if you want to user json, please comment this
# ee.Authenticate()
# # ee.Initialize(project='bukit30project')
# ee.Initialize()

# Initialize the library with json
service_account = 'iqbalpythonapi@bukit30project.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, os.path.join(os.getcwd(),'fcd_gee/bukit30project-4d92e5b46ea7.json'))
ee.Initialize(credentials)

In [4]:
# actual implementation in one row
from asgiref.sync import sync_to_async
from fcd_gee.fcd_run import async_var_assignment, FCDCalc

config = await async_var_assignment('TEST_DEMO_TREEO_delight_planet2022')

classFCD = FCDCalc(config)
fcd_calc_run = classFCD.fcd_calc()

[{'lat': 2.662902245112879, 'lon': 31.747488743897712}, {'lat': 2.659137179189704, 'lon': 31.74820214275822}, {'lat': 2.6578180556469, 'lon': 31.749011000280912}, {'lat': 2.6596104671702805, 'lon': 31.744349156089083}, {'lat': 2.6713688702169387, 'lon': 31.74310343387578}, {'lat': 2.6628167029392897, 'lon': 31.75142743737989}, {'lat': 2.6546868011302656, 'lon': 31.747244138170608}, {'lat': 2.6513804409251995, 'lon': 31.748466062661375}, {'lat': 2.6664971055023052, 'lon': 31.751727781008523}, {'lat': 2.6424465116698377, 'lon': 31.72795793940611}, {'lat': 2.6418155037886817, 'lon': 31.726899513228012}, {'lat': 2.642489912122833, 'lon': 31.725368832567412}, {'lat': 2.645772848365457, 'lon': 31.72654320038452}]
selecting Planet images
selecting Planet images
processing AVI
processing BSI
processing SI
Normalizing to 100 AVI
Normalizing to 100 AVI
Normalizing to 100 BSI
Normalizing to 100 SI
Combining AVI AND BSI
no thermal band, choosing Planet images
Processing means center of AVI_BSI ple

In [5]:
FCD2_1 = fcd_calc_run['FCD2_1']

In [6]:
# AOI = config['AOI']
fcd_selected = config['fcd_selected']

In [7]:
AOIt_shp = os.path.join('./delight_aoi_example.shp')

AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

AOI = AOIt_shp_plot

In [8]:
OID = 'id'
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
            properties= [OID],
            reducer= ee.Reducer.first()
        )

In [9]:
# AOI.getInfo()

In [10]:
print(fcd_selected)

21


In [11]:
# initialize Map object
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

Map.addLayer(FCD2_1,{'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},'FCD_SELECTED')
# Map.addLayer(AOI_img)

In [12]:
from gee_lib.osi.hansen.historical_loss import HansenHistorical

In [13]:
hansen_class = HansenHistorical(config)
tcl = hansen_class.initiate_tcl()
gfc = tcl['gfc']

In [14]:
minLoss = tcl['minLoss']
Map.addLayer(minLoss.clip(AOI),{
            'palette': ['#ff0000']
        }, f'tree cover >30% loss since 20{config["year_start_loss"]}', True)

In [15]:
from gee_lib.osi.classifying.assign_zone import AssignClassZone

In [16]:
assignClass = AssignClassZone(config, FCD2_1 = fcd_calc_run['FCD2_1'], FCD1_1=fcd_calc_run['FCD1_1'],
                                      FCD1_2=fcd_calc_run['FCD1_2'],FCD2_2=fcd_calc_run['FCD2_2'])
class_fcd = assignClass.assigning_fcd_class(gfc, minLoss)

Adding the map of Forest, FCD >= 45% and mask only if not water in area (hansen) and NDWI
Adding the map of Shrubland, FCD <  45% and FCD >= 25%
Adding the map of Grassland or Openland, FCD  < 25%
Processing - the zoning classification
finish processing, merging all the zone into one image


In [17]:
all_zone = class_fcd['all_zone']
vis_params = class_fcd['vis_param_merged']

In [18]:
# image_list = [
#             grassland_gozone, #1
#             shrubland_gozone, #2
#             No_pass_historical_years_rule, #3
#             yrf_forest_dense_10yrs, #4
#             high_forest_dense_10yrs, #5
#             yrf_forest_dense_no10yrs, #6
#             high_forest_dense_no10yrs, #7
#             Water_Un_plantable #8
#         ]



Map.addLayer(all_zone,vis_params,'all_zone')

In [19]:
# AOI.getInfo()

In [20]:
# AOI_client = AOI.getInfo()

In [21]:
# type(AOI_client['features'])

In [22]:
# for i in AOI_client['features']:
#     print(i['properties']['id'])
#     print(i['geometry'])

In [23]:
# geometry_sample = [i['geometry'] for i in AOI_client['features'] if i['properties']['id']==3780][0]

In [24]:
# print(dict_id_zone)

In [25]:
# list_dict_zone_id

In [26]:
# print(list_id)

In [27]:
# Map.addLayer(masked)

In [28]:
from gee_lib.osi.area_calc.main import CalcAreaClass

In [29]:
class_calc_area = CalcAreaClass(config, calc_image=all_zone)
run_calc = class_calc_area.run_calc_per_id()

 executed in 9.03 seconds.
{3779: {'1': 2.72, '2': 4.28, '3': 0, '4': 0, '5': 0, '6': 4.19, '7': 0.71, '8': 0}, 3780: {'1': 1.61, '2': 5.95, '3': 0, '4': 0, '5': 0, '6': 5.08, '7': 0, '8': 0}, 3782: {'1': 0.17, '2': 2.32, '3': 0, '4': 0, '5': 0, '6': 4.85, '7': 0.5, '8': 0}, 3783: {'1': 0.66, '2': 2.79, '3': 0, '4': 0, '5': 0, '6': 5.84, '7': 0.65, '8': 0}, 3800: {'1': 0.5, '2': 2.66, '3': 0, '4': 0, '5': 0, '6': 7.49, '7': 1.83, '8': 0}, 3373: {'1': 0.23, '2': 3.93, '3': 0, '4': 0, '5': 0, '6': 2.75, '7': 0.67, '8': 0}, 3376: {'1': 0.0, '2': 3.57, '3': 0, '4': 0, '5': 0, '6': 4.8, '7': 0, '8': 0}, 3377: {'1': 1.45, '2': 6.16, '3': 0, '4': 0, '5': 0, '6': 2.56, '7': 0.1, '8': 0}, 3392: {'1': 0.45, '2': 1.64, '3': 0, '4': 0, '5': 0, '6': 5.32, '7': 0.21, '8': 0}, 3384: {'1': 0.04, '2': 1.26, '3': 0, '4': 0, '5': 0, '6': 7.01, '7': 0.6, '8': 0}, 3386: {'1': 0, '2': 1.49, '3': 0, '4': 0, '5': 0, '6': 7.99, '7': 0.12, '8': 0}, 3387: {'1': 0.01, '2': 0.39, '3': 0, '4': 0, '5': 0, '6': 7.79,

In [ ]:
# type(AOI)

In [ ]:
# from gee_lib.osi.spectral_indices.utils import assigning_band
# import time

# class CalcAreaClass:
#     def __init__(self, config, calc_image=None):
#         self.config = config
#         self.AOI = config['AOI']
#         scale = None
#         if config['I_satellite'] == 'Sentinel':
#             scale = 10
#         elif config['I_satellite'] == 'Landsat':
#             scale = 30
#         elif config['I_satellite'] == 'Planet':
#             scale = 5
#         self.scale = scale
#         self.AOI_client = self.AOI.getInfo()

#         self.list_id = [i['properties']['id'] for i in self.AOI_client['features']]

#         self.calc_image = calc_image
#         self.calc_pix = ee.Image(assigning_band('pix',1,self.calc_image))
#         # print(list_id)
        
#         self.list_geom = [i['geometry'] for i in self.AOI_client['features']]
#         # print(list_geom)
        
#         self.class_list = ['1','2','3','4','5','6','7','8']
#         # print(class_list)
    
#     # Define calc_area
#     def calc_area_zone(self, class_zone, feature):
#         geometry = feature.geometry()
#         feature_id = feature.get('id')  # hard-coded the field name here is still 'id'
#         # geometry = feature
#         class_mask = self.calc_image.select('Class').eq(ee.Number.parse(class_zone))
        
#         masked_imaged = self.calc_pix.select('pix').mask(class_mask)
    
#         zone_area_image = masked_imaged.multiply(ee.Image.pixelArea()).divide(10000)
        
#         class_pixel_count = zone_area_image.reduceRegion(
#             reducer=ee.Reducer.sum(),
#             geometry=geometry,
#             scale=self.scale
#         ).get('pix')
    
#         result = ee.Feature(geometry, {class_zone: class_pixel_count, 'feature_id': feature_id})
#         # result = {class_zone: class_pixel_count.getInfo()}
#         # result = ee.Feature({class_zone: class_pixel_count})
#         return result

#     def calc_all_zone(self):

#         s = time.perf_counter()
#         # Convert your AOI to an Earth Engine FeatureCollection
#         # aoi_fc = ee.FeatureCollection(AOI_client['features'])
#         aoi_fc = self.AOI
        
#         # geom_list_ee = ee.List(list_geom)
        
#         # Iterate over each class in class_list
#         results = []
#         for class_zone in class_list:
#             # Apply .map to the feature collection for each class
#             class_results = aoi_fc.map(lambda feature: self.calc_area_zone(class_zone, feature))
#             # class_results = geom_list_ee.map(lambda geometry: calc_area_zone(class_zone, geometry))
            
#             # Append the results for the current class to the overall results list
#             results.append(class_results)
        
#         # Convert the results to a Python list
#         final_results = [result.getInfo() for result in results]
        
#         # Print or use final_results as needed
#         # print(final_results)
        
#         elapsed = time.perf_counter() - s
#         print(f" executed in {elapsed:0.2f} seconds.")
#         return final_results
#     def run_calc_per_id(self):
#         # this method to clean the result into a better human readable and api-json serializable
#         list_dict_zone_id = []
#         for i in self.calc_all_zone():
#             for j in i['features']:
#                 # print(j['properties'])
#                 list_dict_zone_id.append(j['properties'])
        
#         # list_id
#         dict_id_zone = {}
#         for i in list_id:
#             init_dict = {}
#             for j in list_dict_zone_id:
                
#                 if j['feature_id'] == i:
#                     for k,v in j.items():
#                         if k != 'feature_id':
#                             init_dict[k] = v
#                 dict_id_zone[i] = init_dict
        
#         # print(dict_id_zone)
        
#         # adjusting dict_id
#         adj_dict_id_zone = {}
#         for k,v in dict_id_zone.items():
#             init_dict = {}
#             for j,l in v.items():
#                 init_dict[j] = round(l,2)
#             adj_dict_id_zone[k] = init_dict
#         print(adj_dict_id_zone)
#         return adj_dict_id_zone

In [ ]:
import asyncio
async def calc_area_zone(class_zone, geometry):
    class_mask = all_zone.select('Class').eq(ee.Number.parse(class_zone))
    class_pixel_count = all_zone_pixel.select('pix').mask(class_mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,  
        scale=scale
    ).get('pix')

    result = {class_zone: class_pixel_count.getInfo()}
    return tuple(result.items())

async def run_calc_zone():
    tasks = [calc_area_zone(class_zone, geometry) 
                   for geometry in list_geom
                        for class_zone in class_list]

    
    # Gather the results of the tasks
    results = await asyncio.gather(*tasks)

    # Convert the list of tuples back to a list of dictionaries
    final_results = [dict(result) for result in results]
    print(final_results)
    return final_results

if __name__ == "__main__":
    s = time.perf_counter()
    
    # In your case, jupyter (IPython ≥ 7.0) is already running an event loop:
    #asyncio.run(main())
    
    await run_calc_zone()
    elapsed = time.perf_counter() - s
    print(f" executed in {elapsed:0.2f} seconds.")

In [ ]:
from pprint import pprint
# pprint(AOI.getInfo())

In [ ]:
from gee_lib.osi.spectral_indices.utils import assigning_band

# Convert class values to binary masks and calculate area for each class
def calculate_class_area(class_value, img, AOI, scale):
    class_mask = img.select('Class').eq(ee.Number.parse(class_value))
    # merged_image = ee.Image(assigning_band('pixel',1,class_mask))
    class_pixel_count = img.select('pixel').mask(class_mask).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=AOI.geometry(),
        scale=scale
    ).get('pixel')

    class_area = ee.Number(class_pixel_count).multiply(scale ** 2).divide(10000)  # Convert square meters to hectares

    # Create a feature representing the class area
    class_feature = ee.Feature(None, {
        'Class': class_value,
        'Area_ha': class_area
    })

    return class_feature

In [ ]:
scale = None
if config['I_satellite'] == 'Sentinel':
    scale = 10
elif config['I_satellite'] == 'Landsat':
    scale = 30
elif config['I_satellite'] == 'Planet':
    scale = 10

In [ ]:
# Get the unique class values in the 'Class' band as an ee.List
class_values_list = ee.Dictionary(
    all_zone.select('Class').reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=AOI.geometry(),
        scale=scale
    ).get('Class')
).keys()

In [ ]:
# Calculate area for each class using the class_values_list
class_areas = class_values_list.map(lambda class_value: calculate_class_area(class_value, all_zone, AOI, scale))

In [ ]:
# Convert the result to a feature collection
result_fc = ee.FeatureCollection(class_areas)

In [ ]:
class_name_map = {
            '2': 'Shrubland_Go-Zone',
            '1': 'Grassland_Go-Zone',
            '7': 'High density Forest',
            '6': 'Low - Medium density Forest',
            '5': 'Regrowth High Density Forest from deforested (historical)',
            '4': 'Regrowth Low Density Forest from deforested (historical)',
            '3': 'Historical deforestation (3 or 10 years rule)',
            '8': 'Water Body'
        }

# Map class names to the DataFrame
result_fc = result_fc.map(lambda f: f.set('Name_Class', class_name_map.get(f.get('Class'))))


In [ ]:
features = result_fc.getInfo()['features']
for feature in features:
    print(feature)

In [ ]:
# Create an empty list to store the data
data = {'Class': [], 'Name_Class': [], 'Area_Ha': []}

# Iterate through the features and extract the properties
features = result_fc.getInfo()['features']
for feature in features:
    properties = feature['properties']

    # Extract the class, class name, and area_ha properties
    class_value = properties['Class']
    name_class = class_name_map.get(class_value, 'Unknown')  # Use 'Unknown' if class_value is not in class_name_map
    area_ha = properties['Area_ha']

    # Append the values to the data dictionary
    data['Class'].append(class_value)
    data['Name_Class'].append(name_class)
    data['Area_Ha'].append(area_ha)

# Create the Pandas DataFrame
import pandas as pd
df = pd.DataFrame(data)

# Optionally, reorder the columns if needed
df = df[['Class', 'Name_Class', 'Area_Ha']]
print(df)

In [ ]:
image_mosaick = fcd_calc_run['image_mosaick']

In [ ]:
Map.addLayer(image_mosaick,{"bands":["red","green","blue"],"min":0,"max":0.1,"gamma":1},'image_mosaick')

In [ ]:
Map

In [ ]:
from asgiref.sync import sync_to_async
from fcd_gee.models import SatVerConfiguration

async def async_filter():
    try:
        label = await sync_to_async(SatVerConfiguration.objects.get)(label="TEST_DEMO_TREEO_Landsat_2022") ## example that already exist in the db and we filterbased on this
    except SatVerConfiguration.DoesNotExist:
        print('label object is not found')
        label = None
    return label

query = await async_filter()

In [ ]:
file_location_conf = await sync_to_async(lambda: query.shp_aoi.file)()

In [ ]:
temp_dir = unzip_file(file_location_conf.path)

In [ ]:
# !cd /tmp/tmpppxot7os && ls

In [ ]:
os.listdir(temp_dir)

In [ ]:
# TEST VARIABLE TO GEE FROM DJANGO
start_date = query.start_date_analysis
end_date = query.end_date_analysis

# for planet labs image
region = 'africa' # or you can use 'asia'

# cloud cover threshold, max per image, in image collection
cloud_cover_threshold = 40

import geemap
# input shp region based on the AOI, relative path to our collab folder content
# AOIt_shp = '/content/drive/MyDrive/shp_local/AOI_AXIS_BIG.shp'

list_files_tmp = os.listdir(temp_dir)
shp_tmp_text = [f for f in list_files_tmp if '.shp' in f][0]
AOIt_shp = os.path.join(temp_dir, shp_tmp_text)

AOIt_shp_plot = geemap.shp_to_ee(AOIt_shp)
crs_input = 'EPSG:4326'

AOI = AOIt_shp_plot
pca_scaling = 1
tileScale = 1


In [ ]:
# choose the satellite image you want to use
# I_satellite = query.satellite_use
I_satellite = 'Planet'

config = {
    'I_satellite': I_satellite,
    'pca_scaling':pca_scaling,
    'tileScale': tileScale,
    'AOI': AOI,
    'IsThermal': False,
    'cloud_cover_threshold': cloud_cover_threshold, # for landsat collection
    'date_start_end':[start_date,end_date],
}

In [ ]:
I_satellite

In [ ]:
#NDWI water limit
ndwi_hi_sentinel = 0.05 # for Sentinel
ndwi_hi_landsat = 0.1 # for landsat
ndwi_hi_planet = -0.2 # for Planet Labs

ndwi_hi = 0.1
if I_satellite == 'Landsat':
    ndwi_hi = ndwi_hi_landsat
elif I_satellite == 'Sentinel':
    ndwi_hi = ndwi_hi_sentinel
elif I_satellite == 'Planet':
    ndwi_hi = ndwi_hi_planet

pca_scaling = 1  # 1 meaning that 1 x pixel size of spatial resolution e.g., Planet Labs 1 x 5, Sentinel 1 x 10, Landsat 1 x 30
tileScale = 1 # increase this if user memory limit occur, see: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

######## HANSEN Tree Cover and Tree Cover Loss Input - Historical Data Check (10 Years Rule) ###################################################
# Hansen 10 years rule and Forest - Tree Cover Hansen
# define tree cover minimum that classified as forest e.g., Indonesia > 30%, hence 30
tree_cover_forest = 30 # this will be in percent, let's say, forest is > 30% if you put 30
pixel_number = 3 #define minimum mapping unit classified as forest, 1 pixel for landsat 30mx30, 3 pixel is forest ~> 0.27 Ha
year_start_loss = 13 #define start year to track as 10 years rule (e.g., 2012 to 2022 (track 10 years rule), hence 12. format= 00->2000, 12->2012)


## FCD Threshold
high_forest = 65
yrf_forest = 45
shrub_grass = 25

# output Band Names
band_name_image = 'Class'

#for area id in shapefile that identified the data, and will converted into raster
OID = 'id'  #IMPORTANT TO CHECK OID based on the column ID
#############################################
##################################################################################
### Masking and overlay and area helper Make an image out of the AOI area attribute -> convert featurecollection into raster (image) for overlaying tools
AOI_img = AOI.filter(ee.Filter.notNull([OID])).reduceToImage(
    properties= [OID],
    reducer= ee.Reducer.first()
)

In [ ]:
from gee_lib.osi.image_collection.main import ImageCollection

In [ ]:
# initiate instance class for the image collection and later mosaicking
classInputCollection = ImageCollection(AOI=AOI, date_start_end=[start_date,end_date], cloud_cover_threshold = cloud_cover_threshold, I_satellite=I_satellite, 
                                       region='africa')


In [ ]:
# performing the merge_collection method based on the I_satellite
image_collection_mask = classInputCollection.image_collection_mask()

In [ ]:
image_mosaick = classInputCollection.image_mosaick()

# initialize Map object
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

In [ ]:
# add to canvas Map for this mosaic layer
if I_satellite == 'Planet':
    # true color {"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}
    # nir veg color {"bands":["red","nir","blue"],"min":0,"max":0.6,"gamma":1.5 }
    Map.addLayer(image_mosaick,{"bands":["red","green","blue"],"min":0,"max":0.6,"gamma":1.5}, f'{I_satellite} mosaicked - {start_date}-{end_date} VegColor')
else:
    Map.addLayer(image_mosaick,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 0.6, 'gamma': 1.5 }, f'{I_satellite} mosaicked - {start_date}-{end_date}')

In [ ]:
Map

In [ ]:
from gee_lib.osi.spectral_indices.spectral_analysis import SpectralAnalysis

In [ ]:
# class image spectral
classImageSpectral = SpectralAnalysis(image_mosaick, config)

In [ ]:
from gee_lib.osi.pca.pca_gee import PCA
from gee_lib.osi.spectral_indices.utils import normalization_100, assigning_band

In [ ]:
print('processing AVI')
avi_image = classImageSpectral.AVI_func()
#Map.addLayer(avi_image,{},'avi')

print('processing BSI')
bsi_image = classImageSpectral.BSI_func()
#Map.addLayer(bsi_image,{},'bsi')

print('processing SI')
si_image = classImageSpectral.SI_func()
#Map.addLayer(si_image,{},'si_no_norm')

#Map.addLayer(image_corr_8,{'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 256,  }, f'landsat_for_FCD_{year}_corrected_8bit')
print('Normalizing to 100 AVI')
##### Starting (normalized 100) Indices
# acquire the existing configuration after conditional I_satellite
pca_scale = classImageSpectral.pca_scale
tileScale = classImageSpectral.tileScale

avi_norm = normalization_100(avi_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(avi_norm,{'min':0,'max':100},'avi_norm', False)
print('Normalizing to 100 BSI')
bsi_norm = normalization_100(bsi_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(bsi_norm,{'min':0,'max':100},'bsi_norm', False)
print('Normalizing to 100 SI')
si_norm = normalization_100(si_image, pca_scale=pca_scale, AOI=AOI)
Map.addLayer(si_norm, {'min':0,'max':100},'si_norm', False)
print('Combining AVI AND BSI')
# Combine  AVI and BSI to one image with two bands
AVI_BSI = avi_norm.addBands(bsi_norm)
# Masked-out process or remove null data, to avoid errors
avi_bsi_clean = AVI_BSI.gte(0).Or(AVI_BSI.lte(0))
AVI_BSI = AVI_BSI.updateMask(avi_bsi_clean)
#Map.addLayer(AVI_BSI, {'min':0,'max':100}, 'AVI_BSI')

print(f'no thermal band, choosing {I_satellite} images')
si = si_norm
#minsi =si.reduceRegion(reducer=ee.Reducer.min(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
#minsi = ee.Dictionary(minsi).toImage()
#maxsi =si.reduceRegion(reducer=ee.Reducer.max(),geometry=AOI,scale=1000,bestEffort=True,tileScale=16)
#maxsi = ee.Dictionary(maxsi).toImage()

#ssiscale = si.subtract(minsi).divide(maxsi.subtract(minsi)).multiply(100)
#print(ssiscale, 'SSI')
Map.addLayer(si,{'min':0, 'max':100}, 'SSI', False)

In [ ]:
sviPCAClass = PCA(AVI_BSI, config)

In [ ]:
sviPCAClass.pca_scale

In [ ]:
print('Processing means center of AVI_BSI please wait')
sviPCA = sviPCAClass.getPrincipalComponents()

In [ ]:
# AOI

In [ ]:
print('Now we proceed to the PCA of Vegetation density')

VD = sviPCA.rename(['VD1', 'VD2'])
SVI = normalization_100(VD,pca_scale=pca_scale, AOI=AOI)

Map.addLayer(SVI,{'min':0,'max':100},'SVI', False)
print('Success get the PCA normalized of VD => SVI')

print('Now calculating the FCD from SVI and SSI - selecting band svi1 svi2 ssi1 and ssi2')
svi1 = SVI.select(['VD1'])
ssi1 = si.select(['SI'])
svi2 = SVI.select(['VD2'])

# Starting to include in the formula for Forest Cover Density
FCD1_1 = (((svi1.multiply(ssi1)).add(1)).pow(0.5)).subtract(1).rename('FCD')
Map.addLayer(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_1')
#Map.addLayer(FCD1.mask(FCD1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_morethan30')

print('Now calculating the FCD from SVI2 and SSI1 into FCD in percentage')
# The best on AOI_B30 based on the visual check, combination svi2 and ssi1
FCD2_1 = (((svi2.multiply(ssi1)).add(1)).pow(0.5)).subtract(1).rename('FCD')

Map.addLayer(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1')
#Map.addLayer(FCD2_1.mask(FCD2_1.gte(30)), {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1_morethan30')
print('finish processing PCA please continue')

In [ ]:
from gee_lib.osi.fcd.main_fcd import FCDCalc

In [ ]:
classFCD = FCDCalc(config)
fcd_all = classFCD.fcd_calc()
FCD2_1 = classFCD.FCD2_1

In [ ]:
# print(FCD2_1)

In [ ]:
# initialize Map object
Map = geemap.Map(center=(-3, 115), zoom=4)
Map.centerObject(AOI, 10)

In [ ]:
# FCD2_1 = fcd_all['FCD2_1']
Map.addLayer(FCD2_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD2_1')

In [ ]:
FCD1_1 = classFCD.FCD1_1
Map.addLayer(FCD1_1, {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 'FCD1_1')

In [ ]:
Map

In [ ]:
classFCD.FCD2_1

In [ ]:
############################ Deforestation areas - TCL (Tree Cover Loss)
### adapted from https://developers.google.com/earth-engine/tutorials/community/forest-cover-loss-estimation
# hansen - updated version - global data
gfc = ee.Image("UMD/hansen/global_forest_change_2022_v1_10")

#Canopy cover percentage (e.g. 30%), for Indonesia
cc = ee.Number(tree_cover_forest)

#Minimum forest area in pixels (e.g. 3 pixels, ~ 0.27 ha in this example).
pixels = ee.Number(pixel_number)

#Minimum mapping area for tree loss (usually same as the minimum forest area).
lossPixels = pixels

canopyCover = gfc.select(['treecover2000'])
canopyCoverThreshold = canopyCover.gte(cc).selfMask()

#Use connectedPixelCount() to get contiguous area.
contArea = canopyCoverThreshold.connectedPixelCount()
#Apply the minimum area requirement.
minArea = contArea.gte(pixels).selfMask()

prj = gfc.projection()
scale = prj.nominalScale()
ForestArea2000Hansen = minArea.reproject(prj.atScale(scale))

#Map.addLayerControl()
Map.addLayer(ForestArea2000Hansen, {
    'palette': ['#96ED89']
}, 'tree cover: >= min canopy cover & area (light green)', False)

#create visual boundary color only
empty = ee.Image().byte()
AOIm = empty.paint(AOI,0,10)

##### Visualized LANDSAT CLOUDLESS FROM HANSEN DATASET LAST YEAR # just to check the visual images from Hansen
LastImageLandsat = gfc.select(['last_b30', 'last_b40', 'last_b50', 'last_b70']).rename(['red', 'nir', 'swir1', 'swir2']).clip(AOI)
Map.addLayer(LastImageLandsat, {'bands': ['swir2', 'nir', 'red'], 'min': 0, 'max': 600, 'gamma': 1.5 },
                f'Landsat Hansen Last Year {end_date}', False)

### HISTORICAL DATA THRESHOLD - Get the TCL data LOSS PIXEL overall data from loss year in the input
# adapted from https://developers.google.com/earth-engine/tutorials/community/forest-cover-loss-estimation

treeLossYear = gfc.select(['lossyear'])
treeLoss = treeLossYear.gte(year_start_loss).selfMask() # tree loss in e.g., year > 2012 ####SHOULD CHANGE TO RECENT YEAR for the '12' number
#Select the tree loss within the derived tree cover
#(>= canopy cover and area requirements). THIS ONE ALREADY MASK TCL IN FOREST AREA
treecoverLoss = minArea.And(treeLoss).rename(f'lossfrom_{year_start_loss}').selfMask()

#Create connectedPixelCount() to get contiguous area.
contLoss = treecoverLoss.connectedPixelCount()
#Apply the minimum area requirement, and get the TCL data ---> minLoss - ACTUAL TCL AREA from Hansen since the year_start_loss
minLoss = contLoss.gte(lossPixels).selfMask()

Map.addLayer(treeLossYear.clip(AOI),{"opacity":1,"bands":["lossyear"],"min":1,"max":22,"palette":["3d358c","4457c9","4777f0","4196ff","2eb4f3","1ad1d5","1ae5b6","36f493","64fd6a","92ff47","b4f836","d3e935","ecd239","fbb938","fe992c","f9751d","ec520e","d93806","bf2102","9f1001","7a0403","#FF0000"]},"Tree Loss Year", False)
Map.addLayer(minLoss.clip(AOI),{
    'palette': ['#ff0000']
}, f'tree cover >30% loss since 20{year_start_loss}', False)


In [ ]:
## CHOOSE WHICH FCD map prefered
FCD = ee.Image(FCD2_1.select(['FCD'])) # here we use comb FCD2_1 = SVI 2, and SI 1 if
Map.addLayer(FCD,{'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']},'FCD_SELECTED')

print('Success calculating FCD best combination, and added to the map')

print('Now we label the FCD')
# Starting to create threshold for labeling
WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
WaterinAOI = AOI_img.mask().updateMask(WaterMask)
# unmasked the water (hansen)
unmaskedWaterAOI = AOI_img.unmask().updateMask(WaterinAOI.mask().Not()).clip(AOI)
#Map.addLayer(unmaskedWaterAOI,{},'unmasked_water_test', False)

######### Zone 5: Water Body - NDWI Sentinel
# from a function, output is already assigned in a band name: Class : 5
#water_list = applying_sentinel_water(image_corrected, ndwi_hi)
#waterinAOI_ndwi = water_list[0]
#unmaskedWaterAOI_ndwi = AOI_img.unmask().updateMask(waterinAOI_ndwi.mask().Not()).clip(AOI)


print(f'Adding the map of Forest, FCD >= {yrf_forest}% and mask only if not water in area (hansen) and NDWI')
#Forest
AllForest = FCD.mask(FCD.gte(yrf_forest)).And(unmaskedWaterAOI)
HighForestDense = FCD.mask(FCD.gte(high_forest)).And(unmaskedWaterAOI)
YRFForestDense = FCD.mask(FCD.gte(yrf_forest).And(FCD.lt(high_forest))).And(unmaskedWaterAOI)
#Map.addLayer(AllForest,{'min':0 ,'max':100}, 'AllForest')
Map.addLayer(HighForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {high_forest}%)', False)

Map.addLayer(YRFForestDense, {'min':0 ,'max':100}, f'Forest (FCD >= {yrf_forest}%) and < {high_forest}%', False)

print(f'Adding the map of Shrubland, FCD <  {yrf_forest}% and FCD >= {shrub_grass}%')
#Shrubland
Shrubland = FCD.mask(FCD.lt(yrf_forest).And(FCD.gte(shrub_grass))).And(unmaskedWaterAOI)
Map.addLayer(Shrubland, {'min':0 ,'max':100}, f'Shrubland ({shrub_grass}% <= FCD < {yrf_forest}%)', False)

print(f'Adding the map of Grassland or Openland, FCD  < {shrub_grass}%')
#Grassland / openland
Grassland = FCD.mask(FCD.lt(shrub_grass)).And(unmaskedWaterAOI)
Map.addLayer(Grassland, {'min':0 ,'max':100}, f'Grassland (FCD < {shrub_grass})', False)

hiforest_masked = AllForest
#Shrubland
#Grassland

print('Processing - the zoning classification')
highForestOnly = ee.Image(assigning_band(band_name_image,1,hiforest_masked.clip(AOI)))
#Map.addLayer(highForestOnly,{'bands': ['Class'],'palette':hi_forest},f'ALL_Forest_FCD > {yrf_forest}% - Class 1')

# Colors fo raster symbology
water = '#3380cc' #Blue
hi_forest = '#006837' #Strong green
low_forest = '#3ea540' #Medium green
grass_land = '#baf096' #Light green
bare_land = '#ad8855' #Brown
other = '#000000'

def applying_zone_sentinel_hansen(hiforest_masked,minLoss,s2_sr_median, ndwi_hi):

    ######### Zone 2 - High Baseline Forest: That does not have (overlaid with) TCL
    # Unmasked the forest loss 10 years rule
    unmaskedLoss = AOI_img.unmask().updateMask(minLoss.mask().Not()).clip(AOI)
    highBaselineF = hiforest_masked.And(unmaskedLoss)
    #Map.addLayer(highBaselineF,{'palette':hi_forest},'High baseline (Forest)')

    # assigning class band 2
    highf_edited = ee.Image(assigning_band(band_name_image,2,highBaselineF)) #high baseline forest - 2
    ##################################################

    ######### Zone 3: High Forest and Loss (e.g., Young Regenerating Forest)
    ####### Get the overlay information of HighBaseline (Sentinel) and Tree Cover Loss (Hansen), e.g., Young Regenerating Forest
    HiForestAndLoss = AOI_img.And(hiforest_masked.And(minLoss)) #minLoss is the actual TCL without overlaying with Sentinel
    #Map.addLayer(HiForestAndLoss,{'palette': '#FF0000'},"High Baseline and not pass 10 years rule")

    # Final Result for High Forest and TCL -> Class number: 3
    tenyrfl_edited = ee.Image(assigning_band(band_name_image,3,HiForestAndLoss)) #tenyears rule not pass and high baseline - 3
    ##############################################

    ######### Zone 4: Tree Cover Loss on the current low baseline (Sentinel)
    ###### Get the 10 years data only that is not overlay with Sentinel High baseline (Forest) #############
    # Create a helper mask indicating where the smaller areas maskhiFL, distinguish only the highbaseline and TCL (mask) and assign mask as 1
    maskHiFL = HiForestAndLoss.mask()
    #Map.addLayer(maskHiFL)
    # Helper - Invert the mask to get the areas where the smaller raster is absent - get only area outside high forest and loss, and mask from 1 to 0
    # invert mask just to get the number from 0 to 1 or vice versa -> .Not()
    maskHiFL_inverted = maskHiFL.Not()
    #Map.addLayer(maskHiFL_inverted)
    # Unmask the bigger raster in the areas to get the pixel value for area 'outside' HiFL (High Baseline and Forest Loss)
    unmaskedHiFL = AOI_img.unmask().updateMask(maskHiFL_inverted).clip(AOI)
    #Map.addLayer(unmaskedHiFL)
    #outcome result for 10 years data only that is not overlay with Sentinel High baseline (Forest) unmaskedHiFL is Area that is not Forest Sentinel############
    tenYearsRule = unmaskedHiFL.And(minLoss)
    #Map.addLayer(tenYearsRule,{'palette': '#FFA500'},"10 Years Rule - not OK")

    # assign the Class into no. 4 - Final Result
    tenyrule_edited = ee.Image(assigning_band(band_name_image,4,tenYearsRule)) #tenyears rule not pass - 4
    ###############################################

    ######### Zone 5: Water Body - NDWI Sentinel
    # from a function, output is already assigned in a band name: Class : 5
    #water_list = applying_sentinel_water(s2_sr_median, ndwi_hi)
    #waterinAOI = water_list[0]
    #water_edited = water_list[1]

    # Use hansen data instead - better result, no need to adjust threshold and water body not change for long time
    WaterMask = gfc.select(['datamask']).rename('datamask').eq(2)
    waterinAOI = AOI_img.mask().updateMask(WaterMask)
    water_edited = ee.Image(assigning_band(band_name_image,5,waterinAOI)).clip(AOI)
    # unmasked the water (hansen)
    unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)

    ###############################################


    ######### Zone 1: Go Zone - outside (forest, forest - TCL, no-forest TCL, WaterBody)
    # Utils - Similar with above (but easier), unmasked the high forest means that in the end only include area that is not in forest (hiforest_masked)
    maskHiF = hiforest_masked.mask()
    maskHiF_inverted = maskHiF.Not()
    # Unmasked High Forest - the result is all the area outside of hiforest_masked (Total all forest), and now in no forest
    unmaskedHiF = AOI_img.unmask().updateMask(maskHiF_inverted).clip(AOI)
    # unmasked the water
    #unmaskedWaterAOI = AOI_img.unmask().updateMask(waterinAOI.mask().Not()).clip(AOI)

    goZone = unmaskedLoss.And(unmaskedHiF).And(unmaskedWaterAOI)
    #Map.addLayer(goZone,{'palette':'#FFFF00'},'Go Zone')
    # assigning band - Go Zone
    goZone_edited = ee.Image(assigning_band(band_name_image,1,goZone)) #go Zone - 1
    ##############################################

    Map.addLayer(goZone_edited,{'bands': ['Class'],'palette':'#FFFF00'},'Go Zone', False)
    Map.addLayer(highf_edited,{'bands': ['Class'],'palette':hi_forest},'High baseline (Forest)', False)
    Map.addLayer(tenyrfl_edited,{'bands': ['Class'],'palette': '#FF0000'},"High Baseline and not pass 10 years rule", False)
    Map.addLayer(tenyrule_edited,{'bands': ['Class'],'palette': '#FFA500'},"10 Years Rule - not OK", False)
    Map.addLayer(water_edited,{'bands': ['Class'],'palette': water},"Water in AOI", False)


    ''' mosaicking into single image  - still does not work
    #image_list = ee.List([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])
    # Extract the spatial information from one of the individual images.
    projection = goZone_edited.projection()
    scale = goZone_edited.projection().nominalScale()

    # Create an ee.ImageCollection from the list of images
    image_collection = ee.ImageCollection([goZone_edited, highf_edited, tenyrfl_edited, tenyrule_edited, water_edited ])

    # Merge the images into a single ee.Image
    zoning_final = image_collection.mosaic()
    '''
    return [goZone_edited,highf_edited,tenyrfl_edited,tenyrule_edited, water_edited]

tiff_result = applying_zone_sentinel_hansen(hiforest_masked,minLoss,image_mosaick, ndwi_hi)

# HighForestDense = FCD.mask(FCD.gte(high_forest).And(unmaskedWaterAOI))
HighForestDense_no10yrs = HighForestDense.And(tiff_result[1]).select(['pixel'])
HighForestDense_no10yrs = ee.Image(assigning_band(band_name_image,21,HighForestDense_no10yrs))

HighForestDense_10yrs = HighForestDense.And(tiff_result[2]).select(['pixel'])
HighForestDense_10yrs = ee.Image(assigning_band(band_name_image,31,HighForestDense_10yrs))

#YRFForestDense = FCD.mask(FCD.gte(yrf_forest).And(unmaskedWaterAOI).And(FCD.lt(high_forest)))
YRFForestDense_no10yrs = YRFForestDense.And(tiff_result[1]).select(['pixel'])
YRFForestDense_no10yrs = ee.Image(assigning_band(band_name_image,22,YRFForestDense_no10yrs))

YRFForestDense_10yrs = YRFForestDense.And(tiff_result[2]).select(['pixel'])
YRFForestDense_10yrs = ee.Image(assigning_band(band_name_image,32,YRFForestDense_10yrs))

Shrubland_gozone = Shrubland.And(tiff_result[0]).select(['pixel'])
Shrubland_gozone = ee.Image(assigning_band(band_name_image,11,Shrubland_gozone))

Grassland_gozone = Grassland.And(tiff_result[0]).select(['pixel'])
Grassland_gozone = ee.Image(assigning_band(band_name_image,12,Grassland_gozone))

# Filter the images based on the 'Class' band
grassland_gozone = Grassland_gozone.select(['Class'])
Map.addLayer(grassland_gozone,{'palette': ['#ffff33']},'Grass land - Go Zone')

shrubland_gozone = Shrubland_gozone.select(['Class'])
Map.addLayer(shrubland_gozone,{'palette': ['#ffe3b3']},'Shrub land - Go Zone')

yrf_forest_dense_no10yrs = YRFForestDense_no10yrs.select(['Class'])
Map.addLayer(yrf_forest_dense_no10yrs,{'palette': ['#83ff5a']},'Low-Med Forest')

high_forest_dense_no10yrs = HighForestDense_no10yrs.select('Class')
Map.addLayer(high_forest_dense_no10yrs,{'palette': ['#09ab0c']},'High- Density Forest')

yrf_forest_dense_10yrs = YRFForestDense_10yrs.select(['Class'])
Map.addLayer(yrf_forest_dense_10yrs,{'palette': ['#ff0abe']},'Re-growth Low-Med Density Forest')

high_forest_dense_10yrs = HighForestDense_10yrs.select(['Class'])
Map.addLayer(high_forest_dense_10yrs,{'palette': ['#ff0004']},'Re-growth High Density Forest')

No_pass_historical_years_rule = tiff_result[3].select(['Class'])
Map.addLayer(No_pass_historical_years_rule,{'palette': ['#ff8a1d']},'Historical deforestation no regrowth (3 or 10 years)')

Water_Un_plantable = tiff_result[4].select(['Class'])
Map.addLayer(Water_Un_plantable,{'palette': ['#1900ff']},'Water body (unplantable)')


empty_image = ee.Image.constant(0).rename('Class')

def add_classes(image, empty):
    # Cast the image to Int32 to handle NaN and null values
    image = image.unmask(0).toInt32()
    # Set a conditional statement to retain existing class values if non-zero, otherwise use the new class value
    merged_image = empty.where(empty.neq(0), empty).where(image.neq(0), image)
    return merged_image

# Create a list of images excluding the 'empty_image'
image_list = [
    grassland_gozone,
    shrubland_gozone,
    yrf_forest_dense_no10yrs,
    high_forest_dense_no10yrs,
    yrf_forest_dense_10yrs,
    high_forest_dense_10yrs,
    No_pass_historical_years_rule,
    Water_Un_plantable
]

# Create an ImageCollection from the list of images
image_collection = ee.ImageCollection(image_list)

# Apply the add_classes function to each image in the collection while merging them into the 'empty_image'
result_collection = image_collection.map(lambda image: add_classes(image, empty_image))

# Create a function to add two images element-wise
def add_images(img1, img2):
    return ee.Image(img1).add(img2)

# Merge all the images in the result_collection using ee.ImageCollection.iterate()
merged_image = ee.Image(result_collection.iterate(add_images, empty_image))

# Cast the merged image to Int32 and set the original Class band name
merged_image = merged_image.toInt32().rename('Class')
merged_image = merged_image.clip(AOI)
print('finish processing')


In [ ]:
# Load DEM data (replace 'dataset' with your actual DEM dataset)
DEM = ee.Image('USGS/SRTMGL1_003').clip(AOI)

# Calculate slope in degrees
slope = ee.Terrain.slope(DEM)

# Convert slope to percentage
slopePercentage = slope.expression('tan(b*0.01745) * 100', {'b': slope})

# Define slope classification thresholds
thresholds = [8, 15, 25, 40]  # Adjust these thresholds as needed

# Classify slope into categories using conditional statements
slopeClasses = slopePercentage \
    .lte(thresholds[0]).multiply(1) \
    .add(slopePercentage.gt(thresholds[0]).And(slopePercentage.lte(thresholds[1])).multiply(2)) \
    .add(slopePercentage.gt(thresholds[1]).And(slopePercentage.lte(thresholds[2])).multiply(3)) \
    .add(slopePercentage.gt(thresholds[2]).And(slopePercentage.lte(thresholds[3])).multiply(4)) \
    .add(slopePercentage.gt(thresholds[3]).multiply(5))

# Display the classified slope image
palette = ['lightgreen', 'yellow', 'orange', 'pink', 'red']  # Change green to lightgreen
vis_params = {'min': 1, 'max': 5, 'palette': palette}
Map.addLayer(slopeClasses, vis_params, 'Slope Classes', True)
# Map.addLayer(DEM)
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(1).selfMask()),{'min':1,'max':100}, 'Grassland Slope 1')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(2).selfMask()),{'min':1,'max':100}, 'Grassland Slope 2')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(3).selfMask()),{'min':1,'max':100},'Grassland Slope 3')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(4).selfMask()),{'min':1,'max':100},'Grassland Slope 4')
#Map.addLayer(Grassland_gozone.And(slopeClasses.eq(5).selfMask()),{'min':1,'max':100},'Grassland Slope 5')

class_name_map = {
    '11': 'Shrubland_Go-Zone',
    '12': 'Grassland_Go-Zone',
    '21': 'High density Forest',
    '22': 'Low - Medium density Forest',
    '31': 'Regrowth High Density Forest from deforested (historical)',
    '32': 'Regrowth Low Density Forest from deforested (historical)',
    '4': 'Historical deforestation (3 or 10 years rule)',
    '5': 'Water Body'
}

key_value_strings = [f'{key}: {value}' for key, value in class_name_map.items()]

class_name_map_color = {
    '11': '#ffe3b3',
    '12': '#ffff33',
    '21': '#09ab0c',
    '22': '#83ff5a',
    '31': '#ff0004',
    '32': '#ff0abe',
    '4': '#ff8a1d',
    '5': '#1900ff'
}

value_list = [value for key, value in class_name_map_color.items()]

labels = key_value_strings
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
colors = value_list
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(title='Legend', labels=labels, colors=colors, position='bottomleft')
Map.addLayerControl()

In [ ]:
# Exporting to tiff
##### INPUT ###########################
title_map = 'Axis Landscape'

AOI_NAME = title_map
START_DATE = start_date
END_DATE = end_date

list_images = [FCD,Grassland_gozone,Shrubland_gozone , YRFForestDense_no10yrs,
                HighForestDense_no10yrs, YRFForestDense_10yrs, HighForestDense_10yrs,
              tiff_result[3], tiff_result[4], slopeClasses]
list_names =  ['FCD','Go Zone_Grassland', 'Go Zone_Shrubland', 'YRF_Forest',
                'High_Dense_Forest','YRF_nopass_historical', 'HDenseForest_nopass_historical',
                'No_pass_historical_years_rule', 'Water_Un_plantable','Slope-Classes']
for i in range(len(list_images)):
    geemap.ee_export_image_to_drive(
        list_images[i], description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_{list_names[i]}', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
    )
    print(f'exported - {list_names[i]}')

# if I_satellite == 'LANDSAT':
#     geemap.ee_export_image_to_drive(
#             image_corrected.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_LANDSAT', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - LANDSAT')

# elif TI_include == False and satellite == 'planet':
#     geemap.ee_export_image_to_drive(
#             filtered_scaled.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_PlanetNICFI_MEDIAN', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - Planet Labs')

# elif TI_include == False and satellite == 'sentinel':
#     geemap.ee_export_image_to_drive(
#             image_corrected.clip(AOI), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_Sentinel_median', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
#         )
#     print(f'exported - Sentinel')

# Print the resulting merged image
#print(merged_image.getInfo())


geemap.ee_export_image_to_drive(
            merged_image.select('Class').toInt(), description=f'{AOI_NAME}_{START_DATE}_{END_DATE}_merged', folder=f"JupyterGEE_{AOI_NAME}", scale=pca_scale, region=AOI.geometry(),
        )
print(f'exported - Merged_classes')

In [ ]:
print(type(conf_selected.start_date_analysis))

In [ ]:
from asgiref.sync import sync_to_async

async def async_filter():
    try:
        label = await sync_to_async(SatVerConfiguration.objects.get)(label="TEST_DEMO_TREEO_Landsat_2022")
    except SatVerConfiguration.DoesNotExist:
        print('label object is not found')
        label = None
    return label

query = await async_filter()

In [ ]:
query.id